In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [ ]:
!pip install openai

GPT-4 ranking. The output of MindMap only choose the summary result (Output1)

In [ ]:
import openai
import csv
from time import sleep

input_file = 'output.csv'
output_file = 'output_ranking.csv'

openai.api_key = "YOUR_OPENAI_KEY"

def prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text):
  prompt_template = """
  Reference: {reference}

  output1: {output1}

  output2: {output2}

  output3: {output3}

  output4: {output4}

  output5: {output5}

  output6: {output6}

  According to the facts of the disease and the drug and test recommendations in reference output, order the fact match of the output from highest to lowest.

  The output followed the format bellow：

  1. Output1
  2. Output5
  3. Output2
  4. Output3
  5. Output4
  6. Output6
  """

  prompt = prompt_template.format(reference=reference_text, output1=output1_text, output2=output2_text, output3=output3_text, output4=output4_text, output5=output5_text, output6=output6_text)
  response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
          {"role": "system", "content": "You are a excellent."},
          {"role": "user", "content": prompt}
      ]
  )

  generated_text = response.choices[0].message.content
  return generated_text

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  header.append("gpt4_ranking")
  writer.writerow(header)

  for row in reader:
    reference_text = [row[1].strip("\n")]
    output1_text = [row[2].strip("\n")]
    output2_text = [row[5].strip("\n")]
    output3_text = [row[6].strip("\n")]
    output4_text = [row[7].strip("\n")]
    output5_text = [row[8].strip("\n")]
    output6_text = [row[9].strip("\n")]

    flag = 0
    while flag == 0:
      try:
        str1 = prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text)
        flag = 1
      except:
        sleep(40)
        str1 = prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text)

    print(str1)
    print('\n')

    row.append(str1)
    writer.writerow(row)




In [ ]:
import re
import csv

input_file = 'output_ranking.csv'
output_file = 'output_ranking_compute.csv'

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  header.extend(["output1_ranking","output2_ranking","output3_ranking","output4_ranking","output5_ranking","output6_ranking"])
  writer.writerow(header)

  for row in reader:
    output_text = row[10]
    print(output_text)
    matches = re.findall(r'(?m)^\d+\. (\w+):', output_text)
    # print("matches1",matches)
    if matches == []:
      matches = re.findall(r'(?m)^\d+\. (\w+)', output_text)
    # print("matches2",matches)
    if matches == []:
      matches = re.findall(r'\d+\.\s+(\w+)', output_text)
    else:
      if matches[0] == "Output":
        matches = re.findall(r'(?m)^\d+\.\s*(Output \d+):', output_text)
        # print('matches3',matches)
        if matches == []:
          matches = re.findall(r'\d+\.\s+(Output\s*\d+)', output_text)

    rankings = {}
    for i, match in enumerate(matches):
      rank = i + 1
      output = match.replace(" ","")
      print(output)
      rankings[output] = rank

    output_dic = {}

    for output, rank in sorted(rankings.items(), key=lambda x: x[1]):
      print(f"{output}: {rank}")
      output_dic[output]=rank



    print(output_dic['Output1'])
    print('\n')

    row.extend([output_dic['Output1'],output_dic['Output2'],output_dic['Output3'],output_dic['Output4'],output_dic['Output5'],output_dic['Output6']])
    writer.writerow(row)


In [ ]:
import pandas as pd

def calculate_column_average(file_path, column_name):
    df = pd.read_csv(file_path)
    average = df[column_name].mean()
    return average

column =["output1_ranking","output2_ranking","output3_ranking","output4_ranking","output5_ranking","output6_ranking"]

file_path = './output_ranking_compute.csv'
for column_name in column:
  average = calculate_column_average(file_path, column_name)
  print(f"The average of column {column_name} is: {average}")